In this notebook we apply some of the methods and code of shape_measurement.ipynb to investigate the effect of pixelization and undersampling on metacalibration.
Important papers:

Huff & Mandelbaum https://arxiv.org/pdf/1702.02600.pdf

Sheldon & Huff    https://arxiv.org/pdf/1702.02601.pdf

In [1]:
#Import libraries
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import galsim
from scipy.optimize import curve_fit
import time
from galsim.gsparams import GSParams
from galsim.hsm import HSMParams
from scipy import fftpack
import shape_measurement as sm

In [2]:
#Simulation options
matchRotNans = True #All rotated images of a galaxy are nan if one of them is nan, if True
nrot = 3 #Number of images per galaxy ie 1 is no rotations, 2 is a 0 and 90 degree rot, etc
ngal = 7 #Number of galaxies to simulate
shearList = [(0,0),(0.04,0),(-0.04,0),(0,0.04),(0,-0.04)] #list of shears to apply
g1shear = [shear[0] for shear in shearList]
g2shear = [shear[1] for shear in shearList]

gsparams = galsim.GSParams(maximum_fft_size=12288)

#Parameters of the galaxy flux distribution
m_min, m_max = 19, 25 #min, max magnitudes
m0 = 25

#Parameters of the galaxy ellipticity function
epsilon0 = 0.35
distortion_cutoff = 0.95
shape_parameter_type = 'distortion'

#Other parameters of the galaxy population
disk_n = 0.5  #n for Sersic profile; 1/2 for Gaussian
disk_re = 0.8 #half-light radius, as

#Parameters of the PSF
psf_sigma = 0.5 #as
pixel_scale = 0.1 #as/px

#Parameters of the noise
noise_on = False
noise_sigma = 0.01 #Gives SNR ~300 for m = 20, m0=25

#Airy PSF
lamda = 750 #nm
diameter = 1.2 #m
airy = galsim.Airy(lam=lamda, diam=diameter, scale_unit=galsim.arcsec, obscuration=0.3)
pixel = galsim.Pixel(pixel_scale)
psf = galsim.Convolve(airy, pixel)
image_psf=airy.drawImage(method='no_pixel')


In [3]:
magArr = np.linspace(m_min,m_max,10000)
mags = sm.draw_samples_specified_distribution(ngal, sm.mag_hist_power_law, magArr, 1)
fluxes = sm.flux(mags, m0)
#Plot histogram
#a=plt.hist(mags, 30, color="k", histtype="step")

In [4]:
distortion = np.random.rayleigh(epsilon0, size=ngal)
#Place cut on distortion as they do in Hoekstra et al 2015
while np.any(distortion>distortion_cutoff):
    high_dist = np.where(distortion>distortion_cutoff)  
    distortion[high_dist] = np.random.rayleigh(epsilon0, size=len(high_dist))
shape_magnitude = distortion

#Generate random numbers for intrinsic galaxy shapes
#shape_params = np.random.random((ngal,2))
phase = np.random.random(ngal)
shape_params = np.stack((shape_magnitude, phase)).transpose()

#Make catalog
#Galaxy ID, Flux/magnitude, size, sersic n, intrinsic ellipticity
galId = range(ngal)
flux = fluxes
size = [disk_re]*ngal
sersic = [disk_n]*ngal
shape1 = list(shape_params[:,0])
shape2 = list(shape_params[:,1] * np.pi)

input_cat = np.stack((galId,flux,size,sersic,shape1,shape2)).transpose()
header = 'galId flux size sersic shape1 shape2 nrot'
np.savetxt('../shear_bias_outputs/input_cat.txt',input_cat,delimiter=' ',header=header)

In [5]:
#load catalog
cat = np.loadtxt('../shear_bias_outputs/input_cat.txt', skiprows=1)
flux = cat[:,1]
disk_reArr, disk_nArr = cat[:,2],cat[:,3]
magArr,phArr = cat[:,4],cat[:,5]

#Generate galaxies
galaxies = [galsim.Sersic(disk_n,half_light_radius=disk_re,gsparams=gsparams) for (disk_re,disk_n) in zip(disk_reArr,disk_nArr)]
galaxies = [gal.withFlux(gal_flux) for gal,gal_flux in zip(galaxies,flux)]
galaxies = [sm.intrinsic_shear(galaxy,shape_parameter_type,mag,ph) for galaxy,mag,ph in zip(galaxies,magArr,phArr)]

#Generate rotated galaxies, add to galaxy list
rotation_angle = [180./(nrot)*i for i in range(1,nrot)]
rot_gals = [gal.rotate(angle*galsim.degrees) for angle in rotation_angle for gal in galaxies]
galaxies += rot_gals

#Shear Galaxies
shearedGals = np.array([[gal.shear(g1=shear[0],g2=shear[1]) for gal in galaxies] for shear in shearList])


In [6]:
startC,startT = time.clock(), time.time()
sim_params = [psf, image_psf, psf_sigma, pixel_scale, disk_reArr, nrot, ngal]
obs, shape_err, error_message = sm.drawGalaxies(shearedGals, -1, True, sim_params)
print 'clock time: %10.1f \n cpu time: %10.1f' % (time.clock()-startC,time.time()-startT)
print 'len(shearedGals) = ', shearedGals.size
print 'Number of errors: ', np.sum(error_message != 0)


clock time:        1.1 
 cpu time:        1.1
len(shearedGals) =  105
Number of errors:  0


In [7]:
weights = np.ones_like(obs)
startC,startT = time.clock(), time.time()
#Find weighted g1 and g2 from e1, e2
meanShear= sm.findMeanShear(obs,weights)
#Use bootstrap method to resample and derive an error on the shear values
size = obs.shape[0]
nBootstrap = int(size*np.log(size)**2)
shearSamples = sm.bootstrapShear(obs,weights,nBootstrap)
errShear = np.std(shearSamples,axis=0) / np.sqrt(nBootstrap)
meanShearBootstrap = np.mean(shearSamples,axis=0)
print 'clock time: %10.1f \n cpu time: %10.1f' % (time.clock()-startC,time.time()-startT)

#Combine mean and err into a single array
res = np.stack((meanShear,errShear)).transpose(2,1,0)
#res Shape: (measurements, shears, (mean, err))

clock time:        0.0 
 cpu time:        0.0


In [8]:
mSamples = sm.bootstrapM(shearSamples,shearList) #shape: numSamples, 2 (g1,g2), SNRs
mStd = np.std(mSamples,axis=0) / np.sqrt(nBootstrap)
mMeanBootstrap = np.mean(mSamples,axis=0)
#mStd shape: ((g1,g2), SNRs)

mMean = sm.getFitParameters(shearList,res)[:,0]

In [9]:
orig_galim = gal.drawImage()
gal_im = galsim.Convolve([gal,psf]).drawImage()

In [10]:
galii = galsim.InterpolatedImage(gal_im)

In [11]:
psfii = galsim.InterpolatedImage(image_psf)

In [12]:
a=sm.cfGal(galii,psfii,shearList)[0].drawImage()